<a href="https://colab.research.google.com/github/Sandwhaletree/Mai_Finlab/blob/main/stk_ch07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-07 年報問答機器人

## 7-2 取得年報

### 1️⃣  匯入套件

In [ ]:
import requests
from bs4 import BeautifulSoup

### 2️⃣ 建立函式-取得年報

In [ ]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  # 建立 POST 請求的表單
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data)
    # 取得回應後擷取檔案名稱
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 建立第二個 POST 請求的表單
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1 # 檔案名稱
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    # 取得 PDF 連結
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 發送 GET 請求
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)
    # 取得 PDF 資料
    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")

### 3️⃣ 呼叫函式

In [ ]:
annual_report('2330','113')

2023_2330_20240604F04.pdf
/pdf/2023_2330_20240604F04_20240614_150441.pdf
OK


## 7-3 年報問答

###4️⃣ 安裝相關套件

由於版本更新的相容性問題, pdfplumber使用舊版本的0.10.2

In [ ]:
!pip install langchain==0.0.335 openai tiktoken pdfplumber==0.10.2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00


###  5️⃣ 匯入相關套件

In [ ]:
import os
import getpass
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

### 6️⃣ 設定環境變數和建立 OpenAI 模型

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
llm_16k = ChatOpenAI(model="gpt-3.5-turbo")

OpenAI API Key:··········


### 7️⃣ 建立函式-建立向量資料庫

In [ ]:
def pdf_loader(file,size,overlap):
  loader = PDFPlumberLoader(file)
  doc = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(
                          chunk_size=size,
                          chunk_overlap=overlap)
  new_doc = text_splitter.split_documents(doc)
  db = FAISS.from_documents(new_doc, OpenAIEmbeddings())
  return db

### 8️⃣  呼叫函式

In [ ]:
db = pdf_loader('/content/113_2330.pdf',500,50)

### 9️⃣ 查詢相關資料

In [ ]:
query = "公司是否有明確的成長或創新策略?"
docs = db.similarity_search(query,k=3)
for i in docs:
    print(i.page_content)
    print('_________')

5.4.1 客戶
提升同仁問題解決與創新的能力，以維持台積公司競爭
優勢並達到客戶滿意的雙贏目標。除了公司內部跨組織 台積公司的客戶遍布全球，產品種類眾多，在半導 最近二年度佔全年度合併營業收入淨額10%以上之客戶資料
的學習交流外，台積公司亦透過「台灣持續改善活動競 體產業的各個領域中表現傑出。客戶包括有無晶
單位：新台幣仟元
賽」，跨產業分享改善手法，期望能以台積公司的經驗分 圓廠設計公司、系統公司和整合元件製造商，例 民國111年 民國110年
名稱
享，促進其他在地產業的發展與進步，並藉由觀摩其他 如：Advanced Micro Devices, Inc.、Amazon Web 佔全年度營業收入 佔全年度營業收入
金額 與發行人之關係 金額 與發行人之關係
淨額比率（%） 淨額比率（%）
產業的改善方法，提升同仁問題解決與創新能力。民國 Services, Inc.、Broadcom Inc.、Intel Corporation、聯
甲客戶 529,649,200 23% 無 405,402,955 26% 無
_________
能對台積公司的營運與財務表現產生負面影響。 台積公司的營運成長及產能的持續擴充計劃，仰賴來自 的營運成本，如果這些專利主張實體成功阻攔本公司所
若台積公司無法克服上述挑戰，則可能對公司業務、財 市場中少數的供應商以得到充分的設備及相關服務，因 提供的產品及服務之交易，亦可能會嚴重干擾本公司的
務狀況及經營結果造成負面影響。 採購集中之風險及因應措施 此可能面臨供應商供給有限且交貨期長的情況，為了優 營運。再者，隨著台積公司的製造營運擴大至部分國外區
●原物料 化供應鏈管理，台積公司針對交貨前置期進行評估及預 域，本公司就智慧財產權被侵害之風險管理亦面臨更高
銷售集中之風險及因應措施 台積公司需及時並以合理的市場價格獲得生產營運所需 測，以期能夠盡量降低供應鏈風險對營運成本的影響， 的挑戰。縱使本公司努力採取有力的措施以減輕在該等
近年來，台積公司的客戶群與客戶的業務性質出現顯著 之足夠原物料，例如矽晶圓、製程用氣體、化學原料、光 台積公司也與各供應商持續擬定多項商務合作模式與 新區域內智慧財產權被侵害的風險，仍無法保證本公司
_________
持其未來發展優勢。
我們將持續致力於優化製造營運（包括「數位化」我們的晶圓廠）來提

### 🔟  匯入問答相關套件

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

### 1️⃣1️⃣  建立函式-問答程式

In [ ]:
# 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,"
     "如果有明確數據或技術(產品)名稱可以用數據或名稱回答,"
     "回答以繁體中文和台灣用語為主。"
     "{context}"),
    ("human","{question}")])

# 建立問答函式
def question_and_answer(question):
    qa = RetrievalQA.from_llm(llm=llm_16k,
                              prompt=prompt,
                              return_source_documents=True,
                              retriever=db.as_retriever(
                                  search_kwargs={'k':10}))
    result = qa(question)
    return result

### 1️⃣2️⃣ 建立迴圈進行問答

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result=question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

輸入問題:公司是否有明確成長?
根據年報中提到的資料，台積公司在民國一百一十一年的營業淨利增加了50%，稅後淨利增加了70.4%，營收增長了42.61%。這些數據顯示公司在該年度實現了明確的成長。此外，公司也提到正在擴大全球製造版圖以增加客戶信任和擴展未來的成長機會。
_________
輸入問題:公司未來一年的計畫
根據所提供的資訊，目前年報並未提供公司未來一年的具體計畫。然而，根據上下文中的一些資訊，可以看出公司未來的一些重點方向和計畫:

1. 優化製造營運：公司將致力於優化製造營運，包括數位化晶圓廠，以提高效率和生產力。

2. 增加全球產能：公司將增加台灣以外地區的產能，以擴大未來的成長潛力，接觸全球人才，並提升客戶信任。

3. 氣候變遷因應：公司將推動低碳製造，邁向淨零排放，並強化氣候韌性，評估重大氣候風險與機會對公司帶來的潛在營運與財務衝擊，並執行相關因應方案。

4. 員工發展與健康：公司將持續關注員工發展，提供培訓課程、健康檢查等福利，並鼓勵員工參與運動活動，建立運動習慣。

需要注意的是，以上僅為根據提供的片段資訊所推斷的一些可能性，具體的未來計畫還需要參考完整的年報資訊。
_________
輸入問題:


## 7-4 年報總結與分析

### 1️⃣3️⃣ 回答結果及原始資料

In [ ]:
from langchain.chains.summarize import load_summarize_chain

### 1️⃣4️⃣ 總結原始資料

In [ ]:
# 建立關鍵字串列
key_word = ['有關市場策略的調整或變化有何提及？',
          '公司對未來一年的展望是什麼？',
          '公司的總收入是否增長，淨利潤的正負情況是否有變化？',
          '國際競爭及海外市場情況',
          '目前的研發狀況?']
data_list = []
for word in key_word:
    data = db.max_marginal_relevance_search(word)
    # 整合 Document 串列
    data_list += data

# 建立提示訊息串列
prompt_template = [("system","你的任務是生成年報摘要。"
                "我們提供年報{text}請你負責生成,"
                "且要保留重點如營收漲跌、開發項目等,"
                "最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

### 1️⃣5️⃣  呼叫函式

In [ ]:
chain_refine_16k = load_summarize_chain(llm=llm_16k,
                                        chain_type='stuff',
                                        prompt=prompt)
print(chain_refine_16k.run(data_list))

根據年報內容，台積公司在過去一年取得了可观的成绩和荣誉。公司營收增加了42.6%，稅后净利润增加了70.4%，每股盈利也有显著提高。在技术方面，公司推出了N4X製程技术、AI智能芯片技术，并持续推进N3和N2的生产计划。此外，公司在可持续发展方面也取得了积极成绩，获得了多项奖项认可，如全球百大创新机构和半导体盛世企业100强等。在社会责任方面，公司积极推动教育、环保和文化传承等项目，助力社会发展。综合来看，台積公司在多个领域取得了显著进展，展现出稳健的发展态势。


### 1️⃣6️⃣  提取關鍵字

In [ ]:
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

word_prompt=ChatPromptTemplate.from_messages(messages=[
    ("human","從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容"
    "")]
)
word_chain = LLMChain(llm=llm_16k, prompt=word_prompt)
output_parser.parse(word_chain('公司的營收狀況如何？')['text'])

['成長', '利潤', '支出', '現金流量']

### 1️⃣7️⃣ 設定 AI 角色讓其分析報告

In [ ]:
data_prompt=ChatPromptTemplate.from_messages(messages=[
    ("system","你現在是一位專業的股票分析師,"
    "你會以詳細、嚴謹的角度進行年報分析, 針對{output}作分析並提及重要數字\
    ,然後生成一份專業的趨勢分析報告。"),
    ("human","{text}")])
data_chain = LLMChain(llm=llm_16k, prompt=data_prompt)

### 1️⃣8️⃣ 整合函式

In [ ]:
def analyze_chain(input):
    # 搜尋「問題」的相關資料
    data = db.max_marginal_relevance_search(input)

    # 第一個 Chain 元件, 建立「關鍵字」串列
    word = word_chain(input)
    word_list = output_parser.parse(word['text'])

    # 搜尋「關鍵字」的相關資料
    for i in word_list:
      data += db.max_marginal_relevance_search(i,k=2)
    word_list.append(input)

    # 第二個 Chain 元件, 生成分析報告
    result = data_chain({'output':word_list,'text':data})

    return result['text']

### 1️⃣9️⃣ 呼叫函式

In [ ]:
input = '公司的營收狀況如何？'
print(analyze_chain(input))

根據台積電的年報資訊，我們可以進行以下方面的分析:

1. 成長：台積電在營業收入淨額增加方面表現出色，主要是由於本年度平均銷售單價提高、晶圓出貨量增加及受匯率變動增加所致。另外，營業費用的增加主要是由於研究發展費用增加所致。

2. 利潤：營業毛利增加主要是平均銷售單價提高、有利的匯率變動及持續成本改善所致，部分受產能利用率下降所抵銷。營業淨利增加主要是因營業毛利增加所致。

3. 現金流：台積電在現金流方面表現出色，全年來自營業活動的現金流入約15,594億，主要包含稅後淨利及折舊、攤銷費用。投資活動的現金流出約9,440億，主要用於資本支出。

4. 市場份額：根據年報資訊，台積電持續精進互補式金氧半導體影像感測器技術，並且正在開發N6射頻技術，以支援車用電子應用，這將有助於增強在市場上的競爭力。

5. 公司的營收狀況：台積電的營收狀況表現出穩健的成長趨勢，主要是由於產品技術的持續進步和市場需求的增加。

總體而言，台積電在成長、利潤和現金流方面呈現穩健的發展趨勢，並且在技術創新和市場份額方面持續投入和努力。這些數字顯示了公司的業務穩健性和成長潛力。
